This notebook produces map-plane plots of the steady-state elevations, flow fields, and melt-rate anomalies... mainly just to remind ourselves that we are dealing with a 3D problem.

In [ ]:
# add path to code
import sys
sys.path.insert(0, '../code')
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.fft import ifft2,fft2
from operators import compute_h, compute_s,h_steady,s_steady,compute_u,compute_v,compute_w
from params import *

sigma = (L/4)/3                                                    # standard deviation for gaussian melt anomaly
m = 5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H      # melt anomaly with amplitude of 5 m/yr

# First compute the elevation solutions for the non-advective background state (positive melt-rate)
u0 = 0 
h_1 = h_steady(k,u0,m)
s_1 = s_steady(k,u0,m)
uh_1 = compute_u(h_1,s_1,k,kx,1)[0,:,:]
vh_1 = compute_v(h_1,s_1,k,ky,1)[0,:,:]
us_1 = compute_u(h_1,s_1,k,kx,0)[0,:,:]
vs_1 = compute_v(h_1,s_1,k,ky,0)[0,:,:]

spd_h1 = np.sqrt(uh_1**2+vh_1**2)
spd_s1 = np.sqrt(us_1**2+vs_1**2)

h_1 = h_1[0,:,:]
s_1 = s_1[0,:,:]

# First compute the elevation solutions for the advective background state (negative melt-rate)
u0 = 2e-1 
h_2 = h_steady(k,u0,-m)
s_2 = s_steady(k,u0,-m)
uh_2 = compute_u(h_2,s_2,k,kx,1)[-1,:,:]
vh_2 = compute_v(h_2,s_2,k,ky,1)[-1,:,:]
us_2 = compute_u(h_2,s_2,k,kx,0)[-1,:,:]
vs_2 = compute_v(h_2,s_2,k,ky,0)[-1,:,:]

spd_h2 = np.sqrt(uh_2**2+vh_2**2)
spd_s2 = np.sqrt(us_2**2+vs_2**2)

h_2 = h_2[-1,:,:]
s_2 = s_2[-1,:,:]

plt.close()

ds = int(6*Nx/100)

levels = np.linspace(-1,1,9)

fig = plt.figure(figsize=(8,9))
plt.subplot(321)

plt.title(r'$\alpha = 0$',fontsize=22)
plt.annotate(r'(a)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
plt.contourf(x0,y0,h_1.T/np.max(np.abs(h_1)),cmap='coolwarm',levels=levels)
plt.axhline(y=0,color='indigo',linestyle='--',linewidth=2)
plt.annotate('Figs.',xy=(-38.5,3),fontsize=16,color='indigo')
plt.annotate('8-10',xy=(-38.5,-9),fontsize=16,color='indigo')
plt.quiver(x0[::ds],y0[::ds],uh_1[::ds,::ds].T/spd_h1.max(),vh_1[::ds,::ds].T/spd_h1.max(),color='k',linewidth=1,units='inches',pivot='mid',scale=4,minlength=0,width=0.02,alpha=0.55)
plt.gca().xaxis.set_ticklabels([])
plt.yticks(fontsize=16)
plt.ylabel(r'$y$',fontsize=20)
plt.xlim(-L,L)
plt.ylim(-L,L)

plt.subplot(322)
plt.annotate(r'(b)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
plt.title(r'$\alpha = 1/5$',fontsize=22)
p1 = plt.contourf(x0,y0,h_2.T/np.max(np.abs(h_2)),cmap='coolwarm',levels=levels)
plt.quiver(x0[::ds],y0[::ds],uh_2[::ds,::ds].T/spd_h2.max(),vh_2[::ds,::ds].T/spd_h2.max(),units='inches',scale=4,pivot='mid',minlength=0,width=0.02,alpha=0.55)
plt.gca().xaxis.set_ticklabels([])
plt.gca().yaxis.set_ticklabels([])
plt.xlim(-L,L)
plt.ylim(-L,L)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.645, 0.02, 0.25])
cbar = fig.colorbar(p1,cax=cbar_ax,orientation='vertical',ticks=levels)
cbar.set_label(label=r'$h\,/\, \Vert h\Vert_\infty$',size=18)
cbar.ax.get_yaxis().labelpad = 10
cbar.ax.tick_params(labelsize=16)


plt.subplot(323)
plt.annotate(r'(c)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
plt.contourf(x0,y0,s_1.T/np.max(np.abs(s_1)),cmap='coolwarm',levels=levels)
plt.quiver(x0[::ds],y0[::ds],us_1[::ds,::ds].T/spd_s1.max(),vs_1[::ds,::ds].T/spd_s1.max(),units='inches',scale=4,pivot='mid',minlength=0,width=0.02,alpha=0.55)
plt.gca().xaxis.set_ticklabels([])
plt.yticks(fontsize=16)
plt.ylabel(r'$y$',fontsize=20)
plt.xlim(-L,L)
plt.ylim(-L,L)

plt.subplot(324)
plt.annotate(r'(d)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
p2 = plt.contourf(x0,y0,s_2.T/np.max(np.abs(s_2)),cmap='coolwarm',levels=levels)
plt.quiver(x0[::ds],y0[::ds],us_2[::ds,::ds].T/spd_s2.max(),vs_2[::ds,::ds].T/spd_s2.max(),units='inches',pivot='mid',scale=4,minlength=0,width=0.02,alpha=0.55)
plt.gca().xaxis.set_ticklabels([])
plt.gca().yaxis.set_ticklabels([])
plt.xlim(-L,L)
plt.ylim(-L,L)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.37, 0.02, 0.25])
cbar = fig.colorbar(p2,cax=cbar_ax,orientation='vertical',ticks=levels)
cbar.set_label(label=r'$s\,/\, \Vert s\Vert_\infty$',size=18)
cbar.ax.get_yaxis().labelpad = 10
cbar.ax.tick_params(labelsize=16)

plt.subplot(325)
plt.annotate(r'(e)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
plt.contourf(x0,y0,m[-1,:,:].T/np.max(np.abs(m)),cmap='coolwarm',levels=levels)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'$x$',fontsize=20)
plt.ylabel(r'$y$',fontsize=20)
plt.xlim(-L,L)
plt.ylim(-L,L)

plt.subplot(326)
plt.annotate(r'(f)',xy=(-38.5,31.5),fontsize=16,bbox=dict(facecolor='w',alpha=1))
p3 = plt.contourf(x0,y0,-m[-1,:,:].T/np.max(np.abs(m)),cmap='coolwarm',levels=levels)
plt.gca().yaxis.set_ticklabels([])
plt.xlabel(r'$x$',fontsize=20)
plt.xticks(fontsize=16)
plt.xlim(-L,L)
plt.ylim(-L,L)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.095, 0.02, 0.25])
cbar = fig.colorbar(p2,cax=cbar_ax,orientation='vertical',ticks=levels)
cbar.set_label(label=r'$m\,/\, \Vert m\Vert_\infty$',size=18)
cbar.ax.get_yaxis().labelpad = 10
cbar.ax.tick_params(labelsize=16)
plt.savefig('map_plane',bbox_inches='tight')
plt.show()
plt.close()
